In [52]:
import pandas as pd
import os
import copy
from typing import List, Sequence, Optional
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from typing import Mapping, Hashable, Optional, Iterable
from plotly.subplots import make_subplots
from statsmodels.formula.api import ols
import re
from backend.cuentas_nacionales.pib import PIBViews
from backend.cuentas_nacionales.pib.pib_constantes import ConstantesPIB as C
from backend.political_terms import tag_politics
from backend.plots import (plot_timeseries_bars, plot_bar_subplots_by_administration, 
                           simple_linegraph, plot_bars, create_pie_chart, 
                           plot_contributions, plot_admin_sector_dumbbell, plot_activity_dumbbell)
from backend.utils import compute_basic_statistics, calculate_component_percentages, validate_percentage_sum, aggregate_columns
from backend.contributions import compute_contributions_pipeline, summary_contributions

In [53]:
g = lambda x: x.map(lambda x:x if type(x) == str else '{:,.2f}'.format(x))

year_on_year_changes = lambda df, q: (df.pct_change(q) * 100).dropna()

In [54]:
# pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [55]:
# colores por presidente:

presidential_colors = {
    "Calderón":   "#9ecae1",  
    "Olsen":      "#fb6a4a",  
    "Rodríguez":  "#fdd0a2",  
    "Pacheco":    "#9467bd",  
    "Arias":      "#74c476",  
    "Chinchilla": "#fdae6b",  
    "Solís":      "#c7c7c7",  
    "Alvarado":   "#e377c2",  
    "Chaves":     "#6baed6",  
}

In [56]:
def create_summary_table(summary):
    """
    Crea una figura de Plotly con la tabla summary.
    
    Espera un DataFrame con columnas:
    ['Actividad', 'Admin_mayor', 'Tasa_mayor', 'Admin_menor', 'Tasa_menor']
    """
    # Copia para no modificar el original
    df = summary.copy()

    # Formatear tasas como porcentaje con 2 decimales
    df["Tasa_mayor"] = df["Tasa_mayor"].map(lambda x: f"{x:.2f}%")
    df["Tasa_menor"] = df["Tasa_menor"].map(lambda x: f"{x:.2f}%")

    # Colores para filas alternas
    fill_colors = []
    for i in range(len(df)):
        fill_colors.append("rgba(245,245,245,1)" if i % 2 == 0 else "white")

    fig = go.Figure(
        data=[
            go.Table(
                header=dict(
                    values=[
                        "<b>Actividad</b>",
                        "<b>Admin. mayor crecimiento</b>",
                        "<b>Tasa mayor</b>",
                        "<b>Admin. menor crecimiento</b>",
                        "<b>Tasa menor</b>",
                    ],
                    fill_color="rgba(30,30,30,1)",
                    font=dict(color="white", size=12),
                    align="center",
                    height=32,
                ),
                cells=dict(
                    values=[
                        df["Actividad"],
                        df["Admin_mayor"],
                        df["Tasa_mayor"],
                        df["Admin_menor"],
                        df["Tasa_menor"],
                    ],
                    align=["left", "center", "right", "center", "right"],
                    fill_color=[fill_colors],
                    font=dict(size=11),
                    height=26,
                ),
                columnwidth=[60, 40, 25, 40, 25],
            )
        ]
    )

    fig.update_layout(
        title=dict(
            text="Resumen de crecimiento promedio por actividad y administración",
            x=0.5,
            xanchor="center",
            font=dict(size=16),
        ),
        margin=dict(l=10, r=10, t=50, b=10),
    )

    return fig

def plot_stats_table_wide(
    stats_by_admin: pd.DataFrame,
    statistic: str = "mean",
    title: str = "Crecimiento interanual promedio del Régimen Definitivo",
    decimals: int = 2,
    width: Optional[int] = 800,
    height: Optional[int] = 220,
    header_color = "salmon"
) -> go.Figure:
    """
    Replica en Plotly la tabla:
        stats_by_admin[[statistic]].transpose()

    Es decir:
    - Una fila (la estadística, p.ej. 'mean')
    - Una columna por administración (Arias, Chinchilla, etc.).
    """

    if statistic not in stats_by_admin.columns:
        raise KeyError(
            f"Statistic '{statistic}' not found in stats_by_admin.columns. "
            f"Available: {list(stats_by_admin.columns)}"
        )

    # Esta línea replica exactamente tu operación:
    df_wide = stats_by_admin[[statistic]].transpose().round(decimals)

    # Columnas = administraciones
    col_labels = df_wide.columns.tolist()
    # Índice = ['mean'] u otra estadística
    row_labels = df_wide.index.tolist()

    # Para go.Table:
    # La primera columna será la etiqueta de la fila ('mean')
    header_vals = [""] + col_labels
    cell_vals = [
        row_labels,                          # primera columna (fila = 'mean')
        *[df_wide[c].tolist() for c in col_labels],  # valores por administración
    ]

    fig = go.Figure(
        data=[
            go.Table(
                header=dict(
                    values=header_vals,
                    fill_color=header_color,
                    font=dict(color="white", size=12),
                    align="center",
                    height=26,
                ),
                cells=dict(
                    values=cell_vals,
                    fill_color=[["#FFFFFF"] * len(row_labels)] * len(header_vals),
                    align="center",
                    format=[None] + [f".{decimals}f"] * len(col_labels),
                    font=dict(color="dimgray", size=11),
                    height=24,
                ),
            )
        ]
    )

    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            xanchor="center",
            font=dict(size=15, color="steelblue"),
        ),
        margin=dict(l=20, r=20, t=60, b=20),
        width=width,
        height=height,
    )

    return fig

# <font color = 'red'> 1. DATOS

In [57]:
folder = r'C:\Users\adolj\OneDrive\Documentos\APPS\informed_economist\data\raw'
file_name = 'Variables_PIB_TC.xlsx'
input_path = os.path.join(folder, file_name)
quarterly_data_levels_TC = pd.read_excel(input_path, index_col= 'fecha')

# Datos en millones de colones
#quarterly_data_levels_TC.info()

In [58]:
opibv = PIBViews(quarterly_data_levels_TC)

## <font color = 'red'> 1.1 RITMO DE CRECIMIENTO DE LA ECONOMÍA

### <font color = 'red'> 1.1 TOTAL DE LA ECONOMÍA

<font color = 'salmon'> ¿En cuáles administraciones la economía costarricense creció más rápido y en cuáles más lento?

In [59]:
# datos en millones de colones
pib = opibv.oferta.get_pib()
# datos en tasas de crecimiento interanuales
pib = year_on_year_changes(pib, q = 4)
# etiqueta de administración
pib = tag_politics(pib)

In [60]:
# datos
df = pib


df.columns = df.columns.str.replace('_TC', '')
col = 'PIB'
statistic = 'mean'
df = df[df['Label'] != 'Calderón']

fig_pib_label = plot_timeseries_bars(
    df=df,
    start = '1994-06-30',
    y_col=col,
    category_col="Label",
    title=f"Costa Rica: Crecimiento Interanual del {col} por Administración",
    x_title="",
    y_title="(%)",
    y_round=2,
    y_suffix="",
    width=850,
    height=420,
    zero_line=False,
    showlegend=True,
    colors=presidential_colors,
    template='plotly_white',
    show_y_grid=True
)

#fig_pib_label.add_annotation(
#    text="Adol J. C.",
#    xref="paper", yref="paper",
#    x=-0.07,  # bien pegado al borde izquierdo
#    y=-0.18,  # bien abajo
#    showarrow=False,
#    font=dict(
#        size=12,                    # tamaño pequeño
#        color="skyblue",   # opacidad baja (35%)
#    ),
#    align="left"
#)

fig_pib_label.show()

stats_by_admin = compute_basic_statistics(df, value_col=col, groupby_col='Label')
stats_by_admin = stats_by_admin.sort_values(by = statistic, ascending = False).round(2)

fig_table = plot_stats_table_wide(
    stats_by_admin=stats_by_admin,
    statistic="mean",
    title="",
    decimals=2,
    width=850,
    height=125,
)

fig_table.show()

Los resultados muestran que las administraciones con mayor dinamismo económico fueron, en primer lugar, la de Óscar Arias Sánchez (2006–2010), con un crecimiento promedio de 4.80%. Le siguen la de Rodrigo Chaves (2022–presente), con 4.51%, y la de Miguel Ángel Rodríguez (1998–2002), con 4.34%. A continuación, se ubican las administraciones de Abel Pacheco (4.25%), Laura Chinchilla (4.18%) y Figueres Olsen (4.12%).

Con un crecimiento más moderado aparece la administración de Luis Guillermo Solís (3.88%), mientras que el desempeño más bajo corresponde a la administración de Carlos Alvarado, con 2.35%, el valor más reducido de todo el periodo analizado.

Aunque el impacto económico de la pandemia coincidió con el gobierno de Carlos Alvarado, la desaceleración no se explica únicamente por ese choque. De hecho, en los trimestres previos al inicio de la crisis sanitaria, el crecimiento promedio era de apenas 2.24%, lo que evidencia que la economía ya mostraba un debilitamiento importante antes del COVID-19, y más bien.


In [61]:
# orden canónico
canonical_order = stats_by_admin.index # admin_order

In [62]:
#canonical_order = ['Arias', 'Chaves', 'Rodríguez', 'Pacheco',
#                   'Chinchilla', 'Olsen', 'Solís', 'Alvarado']

canonical_order_2 = copy.deepcopy(list(canonical_order))
canonical_order_2.remove('Olsen') 
canonical_order_2.remove('Rodríguez') 
canonical_order_2.remove('Pacheco') 

In [63]:
# round(pib[(pib['Label'] == 'Alvarado')].loc[:'2020-03-31']['PIB'].mean(), 2)

### <font color = 'red'> 1.2 RÉGIMEN DE COMERCIO (RÉGIMEN DEFINITIVO Y ZONAS FRANCAS)

In [64]:
# datos en millones de colones
pib_regimen = opibv.dataframe[['PIB_RegDef_TC', 'PIB_RegEsp_TC']]
# datos en tasas de crecimiento interanuales
pib_regimen = year_on_year_changes(pib_regimen, q = 4)
# etiqueta de administración
pib_regimen = tag_politics(pib_regimen)

### <font color = 'red'> 1.2.1 RÉGIMEN DEFINITIVO

In [65]:
# datos
df = pib_regimen[['PIB_RegDef_TC', 'Label']]


df.columns = df.columns.str.replace('_TC', '')
col = 'PIB_RegDef'
statistic = 'mean'
df = df[df['Label'] != 'Calderón']

fig_pib_label = plot_timeseries_bars(
    df=df,
    start = '1994-06-30',
    y_col=col,
    category_col="Label",
    title=f"Costa Rica: Crecimiento Interanual del Régimen Definitivo por Administración",
    x_title="",
    y_title="(%)",
    y_round=2,
    y_suffix="",
    width=850,
    height=420,
    zero_line=False,
    showlegend=True,
    colors=presidential_colors,
    template='plotly_white',
    show_y_grid=True
)

#fig_pib_label.add_annotation(
#    text="Adol J. C.",
#    xref="paper", yref="paper",
#    x=-0.07,  # bien pegado al borde izquierdo
#    y=-0.18,  # bien abajo
#    showarrow=False,
#    font=dict(
#        size=12,                    # tamaño pequeño
#        color="skyblue",   # opacidad baja (35%)
#    ),
#    align="left"
#)

fig_pib_label.show()

stats_by_admin = compute_basic_statistics(df, value_col=col, groupby_col='Label')
stats_by_admin = stats_by_admin.sort_values(by = statistic, ascending = False).round(2)

fig_table = plot_stats_table_wide(
    stats_by_admin=stats_by_admin,
    statistic="mean",
    title="",
    decimals=2,
    width=850,
    height=125,
)

fig_table.show()




In [66]:
round(df[(df['Label'] == 'Alvarado')].loc[:'2020-03-31'][col].mean(), 2)

np.float64(1.53)

Las administraciones con mayor dinamismo en el Régimen Definitivo fueron las de Óscar Arias, con un crecimiento promedio de 4.86%, seguidas por Laura Chinchilla (4.19%), Abel Pacheco (4.09%) y Miguel Ángel Rodríguez (4.00%).

Con un desempeño más moderado se ubican las administraciones de José María Figueres Olsen (3.84%), Rodrigo Chaves (3.60%) y Luis Guillermo Solís (3.06%).

Finalmente, el crecimiento más bajo corresponde, nuevamente, a la administración de Carlos Alvarado, con un promedio de 1.38%. Incluso al considerar únicamente los trimestres previos a la pandemia, el dinamismo económico se mantenía débil: el crecimiento interanual promedio en ese periodo fue de apenas 1.53%, lo que confirma que la desaceleración ya estaba presente antes del impacto del COVID-19.

### <font color = 'red'> 1.2.2 ZONAS FRANCAS

In [67]:
# datos
df = pib_regimen[['PIB_RegEsp_TC', 'Label']]


df.columns = df.columns.str.replace('_TC', '')
col = 'PIB_RegEsp'
statistic = 'mean'
df = df[df['Label'] != 'Calderón']

fig_pib_label = plot_timeseries_bars(
    df=df,
    start = '1994-06-30',
    y_col=col,
    category_col="Label",
    title=f"Costa Rica: Crecimiento Interanual del Régimen Especial por Administración",
    x_title="",
    y_title="(%)",
    y_round=2,
    y_suffix="",
    width=850,
    height=420,
    zero_line=False,
    showlegend=True,
    colors=presidential_colors,
    template='plotly_white',
    show_y_grid=True
)

#fig_pib_label.add_annotation(
#    text="Adol J. C.",
#    xref="paper", yref="paper",
#    x=-0.07,  # bien pegado al borde izquierdo
#    y=-0.18,  # bien abajo
#    showarrow=False,
#    font=dict(
#        size=12,                    # tamaño pequeño
#        color="skyblue",   # opacidad baja (35%)
#    ),
#    align="left"
#)

fig_pib_label.show()

stats_by_admin = compute_basic_statistics(df, value_col=col, groupby_col='Label')
stats_by_admin = stats_by_admin.sort_values(by = statistic, ascending = False).round(2)

fig_table = plot_stats_table_wide(
    stats_by_admin=stats_by_admin,
    statistic="mean",
    title="",
    decimals=2,
    width=850,
    height=125,
)

fig_table.show()


Por su parte, las administraciones con mayor dinamismo en las Zonas Francas fueron las de Luis Guillermo Solís (15.78%), Figueres Olsen (15.67%), Rodríguez (12.46%), Rodrigo Chaves (10.77%) y Carlos Alvarado (10.76%).

En menor medida se ubican las administraciones de Pacheco (7.38%), Oscar Arias (4.20%) y Laura Chinchilla (3.98%).

### <font color = 'red'> 1.2 OFERTA: ACTIVIDAD ECONÓMICA

<font color = 'salmon'> ¿Cuál Administración creció más y cuál menos en cada una de las actividades económicas?

La administración de Miguel Ángel Rodríguez destaca por ser la que registró el mayor número de actividades económicas con el crecimiento promedio más alto, específicamente en Información y Comunicaciones (19.04%), Actividades Profesionales (13.14%), Electricidad, Agua y Saneamiento (6.92%), y Minas y Canteras (4.49%), donde obtuvo el mejor desempeño relativo frente a las demás administraciones. En contraste, la administración de Carlos Alvarado es la que aparece con mayor frecuencia como la de menor crecimiento promedio, ubicándose en la última posición en sectores como Construcción (–2.47%), Minas y Canteras (–1.11%), Agricultura (0.22%), Inmobiliario (1.30%), Enseñanza, Salud y Asistencia Social (2.14%), Información y Comunicaciones (3.43%) y Actividades Profesionales (4.30%).

In [68]:
# datos en millones de colones
pib_ind = opibv.oferta.get_industrias()
# datos en tasas de crecimiento interanuales
pib_ind = year_on_year_changes(pib_ind, q = 4)
# etiqueta de administración
pib_ind = tag_politics(pib_ind)

# display(g(pib_ind.tail(3)))

In [69]:
df = pib_ind

# 1. Columnas económicas
cols_econ = [c for c in df.columns if c.startswith("PIB_")]

# 2. Agrupar por administración y calcular crecimiento promedio
growth_by_admin = df.groupby("Label")[cols_econ].mean()

# 3. Reordenar en orden canónico

growth_by_admin = growth_by_admin.loc[canonical_order]

# 4. Identificación de mayor y menor crecimiento por actividad
admin_max = growth_by_admin.idxmax()
admin_min = growth_by_admin.idxmin()

val_max = growth_by_admin.max()
val_min = growth_by_admin.min()

# 5. Construcción del resumen completo
summary = pd.DataFrame({
    "Actividad": cols_econ,
    "Admin_mayor": admin_max.values,
    "Tasa_mayor": val_max.values,
    "Admin_menor": admin_min.values,
    "Tasa_menor": val_min.values
})

fig_summary = create_summary_table(summary)
fig_summary.show()


* En Agricultura, Silvicultura y Pesca, la administración Olsen registró el mayor crecimiento promedio (5.85%), mientras que Alvarado mostró el desempeño más débil (0.22%).

* En Minas y Canteras, el mayor dinamismo correspondió a Rodríguez (4.49%), en contraste con Alvarado, que presentó una caída promedio de –1.11%.

* Para Manufactura, la administración Chaves lideró con 6.91%, mientras que Arias tuvo el menor resultado (–0.39%).

* En Electricidad, Agua y Saneamiento, la administración de Rodríguez destacó con 6.92%, siendo Chinchilla la administración con menor desempeño (–1.13%).

* El sector Construcción tuvo su punto máximo durante el gobierno de Arias (9.75%) y el mínimo con Alvarado (–2.47%).

* En Comercio, la administración Chinchilla logró el mayor crecimiento promedio (5.37%), mientras que Rodríguez registró el más bajo (0.69%).

* En Transporte y Almacenamiento, el mejor resultado se observó bajo Chaves (6.75%) y el peor durante Olsen (–2.77%).

* El sector Hoteles y Restaurantes alcanzó su mayor dinamismo con Chinchilla (7.08%), mientras que Olsen mostró la menor expansión (2.14%).

* En Información y Comunicaciones, sobresale el crecimiento excepcional bajo Rodríguez (19.04%), mientras que Alvarado tuvo la tasa más baja (3.43%).

* En Financieras y Seguros, la mayor expansión ocurrió durante la administración Arias (14.38%), en contraste con la de Chaves (3.93%).

* El sector Inmobiliario alcanzó su punto más alto con Olsen (12.63%) y el más bajo con Alvarado (1.30%).

* En Actividades Profesionales, la administración Rodríguez lideró con 13.14%, siendo Alvarado la de menor resultado (4.30%).

* Para Administración Pública, la mayor tasa provino de Arias (3.28%), mientras que Olsen registró el valor mínimo (0.10%).

* En Enseñanza, Salud y Asistencia Social, Olsen nuevamente tuvo el mejor desempeño (4.06%) frente a Alvarado, que obtuvo la menor tasa (2.14%).

*  Finalmente, en Otras Actividades, el liderazgo correspondió a Chinchilla (6.77%) y el menor crecimiento se registró en la administración Olsen (–0.72%).

In [70]:
# Partimos del summary "largo" que tenías:
# columnas: Actividad, Admin_mayor, Tasa_mayor, Admin_menor, Tasa_menor

summary_act = summary.copy()

# Si querés usar SHORT_NAMES en el índice:
summary_act["Actividad_short"] = summary_act["Actividad"].map(C.SHORT_NAMES).fillna(summary_act["Actividad"])
summary_act = summary_act.set_index("Actividad_short")

# Dejamos solo las columnas necesarias
summary_act = summary_act[["Admin_mayor", "Tasa_mayor", "Admin_menor", "Tasa_menor"]]

fig_act = plot_activity_dumbbell(
    summary=summary_act,
    presidential_colors=presidential_colors,
    x_range=(-8, 24),
    height=610,
    title_color="steelblue",
    axis_label_color="gray",
    tick_color="black",
    legend_color="black",
    line_color="#1f77b4",
    line_width=4,
)

fig_act.update_layout(
    legend_font=dict(color="steelblue", size=12)
)

fig_act.show()

In [71]:
# datos
df = pib_ind

df.info()

print('\n')

#df.columns = df.columns.str.replace('_TC', '')
col = 'PIB_Ense_Salud_Asistencia_Social_TC'
statistic = 'mean'
df = df[df['Label'] != 'Calderón']

fig_pib_label = plot_timeseries_bars(
    df=df,
    start = '1994-06-30',
    y_col=col,
    category_col="Label",
    title=f"Costa Rica: Crecimiento Interanual del {col} por Administración",
    x_title="",
    y_title="(%)",
    y_round=2,
    y_suffix="",
    width=850,
    height=420,
    zero_line=False,
    showlegend=True,
    colors=presidential_colors,
    template='plotly_white',
    show_y_grid=True
)

fig_pib_label.show()

stats_by_admin = compute_basic_statistics(df, value_col=col, groupby_col='Label')
stats_by_admin = stats_by_admin.sort_values(by = statistic, ascending = False).round(2)

fig_table = plot_stats_table_wide(
    stats_by_admin=stats_by_admin,
    statistic="mean",
    title="",
    decimals=2,
    width=850,
    height=125,
)

fig_table.show()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 135 entries, 1992-03-31 to 2025-09-30
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   PIB_Agricultura_Silvicultura_Pesca_TC  135 non-null    float64
 1   PIB_Minas_Canteras_TC                  135 non-null    float64
 2   PIB_Manufactura_TC                     135 non-null    float64
 3   PIB_Electricidad_Agua_Saneamiento_TC   135 non-null    float64
 4   PIB_Construccion_TC                    135 non-null    float64
 5   PIB_Comercio_TC                        135 non-null    float64
 6   PIB_Transporte_Almacenamiento_TC       135 non-null    float64
 7   PIB_Hoteles_Restaurantes_TC            135 non-null    float64
 8   PIB_Informacion_Comunicaciones_TC      135 non-null    float64
 9   PIB_Financieras_Seguros_TC             135 non-null    float64
 10  PIB_Inmobiliario_TC                    135 non-null    

<font color = 'salmon'> ¿Cuál actividad económica creció más y cuál menos en cada administración?

In [72]:
df = pib_ind

# 1) Promedio de cada actividad por administración
mean_by_admin = df.drop(['President', 'Party', 'Term'], axis = 1).groupby("Label").mean() 

# 2) Para cada administración: actividad con mayor crecimiento y menor crecimiento
summary = pd.DataFrame({
    "Max_Activity": mean_by_admin.idxmax(axis=1),
    "Max_Growth":   mean_by_admin.max(axis=1),
    "Min_Activity": mean_by_admin.idxmin(axis=1),
    "Min_Growth":   mean_by_admin.min(axis=1),
})

summary = summary.loc[canonical_order]

g(round(summary, 2))


,Max_Activity,Max_Growth,Min_Activity,Min_Growth
Label,,,,
Arias,PIB_Financieras_Seguros_TC,14.38,PIB_Manufactura_TC,-0.39
Chaves,PIB_Actividades_Profesionales_TC,7.97,PIB_Administracion_Publica_TC,0.37
Rodríguez,PIB_Informacion_Comunicaciones_TC,19.04,PIB_Comercio_TC,0.69
Pacheco,PIB_Informacion_Comunicaciones_TC,18.42,PIB_Administracion_Publica_TC,1.23
Chinchilla,PIB_Informacion_Comunicaciones_TC,11.46,PIB_Construccion_TC,-2.00
Olsen,PIB_Inmobiliario_TC,12.63,PIB_Transporte_Almacenamiento_TC,-2.77
Solís,PIB_Informacion_Comunicaciones_TC,10.00,PIB_Administracion_Publica_TC,1.14
Alvarado,PIB_Manufactura_TC,6.90,PIB_Construccion_TC,-2.47


In [73]:
summary_pretty = summary.copy()
summary_pretty["Max_Activity"] = summary_pretty["Max_Activity"].map(C.SHORT_NAMES).fillna(summary_pretty["Max_Activity"])
summary_pretty["Min_Activity"] = summary_pretty["Min_Activity"].map(C.SHORT_NAMES).fillna(summary_pretty["Min_Activity"])

fig = plot_admin_sector_dumbbell(
    summary_pretty,
    title_color = 'steelblue',
    axis_label_color  = 'gray',
    presidential_colors = presidential_colors,
    admin_order=canonical_order,
    x_range=(-7, 22),
    
    
)
fig.update_layout(autosize=True)

#fig.add_annotation(
#    text="Adol J. C.",
#    xref="paper", yref="paper",
#    x=-0.12,  # bien pegado al borde izquierdo
#    y=-0.07,  # bien abajo
#    showarrow=False,
#    font=dict(
#        size=12,                    # tamaño pequeño
#        color="skyblue",   # opacidad baja (35%)
#    ),
#    align="left"
#)

fig.update_layout(
    legend_font=dict(color="steelblue", size=12)
)

fig.update_xaxes(zeroline=False)



fig.show()

Durante la administración de Óscar Arias (2006–2010), el mayor crecimiento provino del sector de Servicios Financieros y Seguros, con un incremento promedio cercano al 14.4%. En contraste, Manufactura registró la menor variación, con una ligera caída del –0.39%.

En la administración de Rodrigo Chaves (2022–presente), la actividad más dinámica ha sido Actividades Profesionales, con un crecimiento del 7.97%, mientras que Administración Pública mostró el menor avance, aunque positivo, con 0.37%.

Durante el gobierno de Miguel Ángel Rodríguez (1998–2002), el principal motor económico fue Información y Comunicaciones, con un notable crecimiento del 19.04%. Por su parte, la actividad con menor dinamismo fue Comercio, con un incremento moderado de 0.69%.

En la administración de Abel Pacheco (2002–2006), también destacó Información y Comunicaciones como la actividad de mayor crecimiento, con 18.42%, mientras que Administración Pública nuevamente figuró como el sector menos dinámico (1.23%).

Bajo la administración de Laura Chinchilla (2010–2014), Información y Comunicaciones lideró con un crecimiento del 11.46%, mientras que Construcción presentó la mayor contracción, con –2.00%.

En el periodo de José María Figueres Olsen (1994–1998), la actividad más dinámica fue Inmobiliario, con un crecimiento del 12.63%, en tanto que Transporte y Almacenamiento experimentó la mayor caída del conjunto, con –2.77%.

Durante el gobierno de Luis Guillermo Solís (2014–2018), Información y Comunicaciones volvió a ser la actividad con mayor crecimiento (10.00%), mientras que Administración Pública registró la menor variación (1.14%).

Finalmente, en la administración de Carlos Alvarado (2018–2022), Manufactura fue la actividad con mejor desempeño, con un crecimiento del 6.90%, mientras que Construcción presentó una contracción significativa de –2.47%.

En conjunto, estos resultados muestran que si bien ciertas actividades —como Información y Comunicaciones— han liderado de forma recurrente en múltiples administraciones, cada período exhibe una dinámica particular respecto a cuál sector actuó como motor principal y cuáles se rezagaron.

### <font color = 'red'> 1.3 OFERTA: SECTORES

<font color = 'salmon'> ¿Cuál Administración creció más y cuál menos por Sector Productivo?

In [74]:
# datos en millones de colones
pib_sectores = opibv.oferta.get_sectores()
# datos en tasas de crecimiento interanuales
pib_sectores = year_on_year_changes(pib_sectores, q = 4)
# etiqueta de administración
pib_sectores = tag_politics(pib_sectores)

In [75]:
df = pib_sectores

df.columns = df.columns.str.replace('_TC', '')
df.rename(columns={'Industria_Ampliada': 'Industria'}, inplace = True)
df = df.drop(['President', 'Party', 'Term'], axis = 1)

means = df.groupby("Label").mean()
to_remove = ["Calderón"]
means = means.drop(index=to_remove, errors="ignore")


fig = plot_bar_subplots_by_administration(df = means, variables = means.columns, colors = presidential_colors, title = "Costa Rica: Crecimiento Promedio del PIB por Administración y Sector", y_title = "Crecimiento Interanual (%)")
fig.show()

#g(means.loc[canonical_order])

•	En el sector agropecuario, la administración de Figueres Olsen registra el mayor crecimiento promedio, con un 5.85%, claramente por encima del resto. Le siguen las administraciones de Abel Pacheco (3.19%), Laura Chinchilla (2.72%) y Luis Guillermo Solís (2.19%), todas mostrando desempeños más moderados, superiores al 2% promedio. Más rezagadas aparecen las administraciones de Miguel Ángel Rodríguez (1.92%) Óscar Arias (1.77%), Rodrigo Chaves (0.75%) y, en último lugar, Carlos Alvarado (0.22%).

•	En Servicios, la administración de Óscar Arias sobresale con el crecimiento promedio más alto, alcanzando 6.04%. Le siguen Laura Chinchilla (5.20%), Luis Guillermo Solís y Miguel Ángel Rodríguez, ambos con 4.58%, Rodrigo Chaves (4.43%), Abel Pacheco (4.34%). Los menores crecimientos corresponden a José María Figueres Olsen (3.05%) y, más rezagado aún, Carlos Alvarado (1.90%).

•	En el sector industrial, el mayor crecimiento promedio lo registra la administración de Rodrigo Chaves, con un crecimiento interanual promedio de 5.52%. Le siguen Carlos Alvarado (4.00%), Miguel Ángel Rodríguez (3.89%), Abel Pacheco (3.53%), y Figueres Olsen (3.52%). Más atrás se ubican Óscar Arias (2.35%), Luis Guillermo Solís (2.11%) y Laura Chinchilla (1.21%), estos últimos reflejando una menor expansión industrial.


### <font color = 'red'> 1.4 DEMANDA: COMPONENTES DEL GASTO

<font color = 'salmon'> ¿Cuál Administración creció más y cuál menos por Componente del Gasto?

In [77]:
# datos en millones de colones
pib_comp_gasto = opibv.demanda.get_categorias_principales()
# datos en tasas de crecimiento interanuales
pib_comp_gasto = year_on_year_changes(pib_comp_gasto, q = 4)
# etiqueta de administración
pib_comp_gasto = tag_politics(pib_comp_gasto)

pib_comp_gasto.drop('PIB_TC', axis = 1, inplace = True)

In [80]:
df =  pib_comp_gasto
df.columns = df.columns.str.replace('_TC', '')


s = {'PIB': 'PIB',
     'PIB_Gasto_Consumo_Final_Hogares': 'Consumo Hogares',
     'PIB_Gasto_Consumo_Final_Gobierno_General': 'Consumo Gobierno',
     'PIB_Formacion_Bruta_Capital_Fijo': 'Inversión',
     'PIB_Exportaciones_Bienes_Servicios': 'Exportaciones',
     'PIB_Importaciones_Bienes_Servicios': 'Importaciones'}

df.rename(columns=s, inplace = True)

components = ["Consumo Hogares", "Consumo Gobierno", "Inversión", "Exportaciones", "Importaciones"]
means = df.groupby("Label")[components].mean()
to_remove = ["Calderón"]
means = means.drop(index=to_remove, errors="ignore")

means

,Consumo Hogares,Consumo Gobierno,Inversión,Exportaciones,Importaciones
Label,,,,,
Alvarado,1.326384,2.107886,0.050918,4.683875,2.253635
Arias,4.645113,4.453843,5.827179,2.633044,1.764197
Chaves,3.724922,1.334993,3.976080,10.081315,5.883434
Chinchilla,5.562635,2.349667,5.705397,5.715995,8.937741
Olsen,3.386958,1.565888,8.249487,8.258742,7.624276
Pacheco,4.466159,1.573161,3.675952,6.846381,6.051212
Rodríguez,2.366070,2.614695,4.028183,4.813029,1.486473
Solís,4.602864,2.420659,3.362131,6.186144,5.782604


In [83]:
fig = plot_bar_subplots_by_administration(df = means, variables = components, 
                                          colors = presidential_colors, 
                                          title = "PIB: Crecimiento por Administración y por Componentes de la Demanda", 
                                          y_title = "Crecimiento Interanual (%)",
                                          x_title = 'Posiciones',
                                          fig_width = 1100,
                                          )
fig.show()


g(means.loc[canonical_order])   

,Consumo Hogares,Consumo Gobierno,Inversión,Exportaciones,Importaciones
Label,,,,,
Arias,4.65,4.45,5.83,2.63,1.76
Chaves,3.72,1.33,3.98,10.08,5.88
Rodríguez,2.37,2.61,4.03,4.81,1.49
Pacheco,4.47,1.57,3.68,6.85,6.05
Chinchilla,5.56,2.35,5.71,5.72,8.94
Olsen,3.39,1.57,8.25,8.26,7.62
Solís,4.60,2.42,3.36,6.19,5.78
Alvarado,1.33,2.11,0.05,4.68,2.25
